Main Questions:

- Which U.S. state has the best market conditions for a new restaurant or takeout business?

- What type of restaurant (eg., fast food, casual dining, fine dining, ethnic cuisine) has the highestr success rate?

- What demographic, economic and consumer behavior factors influence restaurant success?

US Population Data (By State) ~ 2020 to 2024

In [2]:
#importing necessary libraries

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import json
from serpapi import GoogleSearch
from tqdm import tqdm

In [ ]:
#Loading the dataset
df = pd.read_csv('Datasets/Population Data (2020-2024).csv')

df.head()

In [ ]:
#Dropping unnecessary rows

#df = df.drop(index=list(range(0, 13)))
#df = df.drop(index = 13)

df.head()

In [ ]:
#Plotting the population by state and selecting the top 5 states

#Extracting state names and population estimates for 2024
states = df["NAME"]
pop_est = df["POPESTIMATE2023"]
top_states = pop_est.nlargest(5).index #index of top 5 states
toppop_est = pop_est[top_states] #Population values of top 5 states
top_states_names = states[top_states] #State names

#plot the data
plt.figure(figsize=(12, 6))
plt.barh(top_states_names, toppop_est, color="skyblue")
plt.xlabel("Population Estimate (2023)")
plt.ylabel("State")
plt.title("Population Estimates for US States in 2023")
plt.gca().invert_yaxis()  # Invert y-axis for better readability
plt.show()

In [ ]:
#Top 5 States with Highest Consumer Spending Amounts

cs_df = pd.read_excel('Datasets/pce1024.xlsx')

cs_df.head()

In [ ]:
# Load the dataset for all three states (replace with your file paths)
california_df = pd.read_csv('Datasets/SAINC70_CA_2000_2023.csv')
texas_df = pd.read_csv('Datasets/SAINC70_TX_2000_2023.csv')
florida_df = pd.read_csv('Datasets/SAINC70_FL_2000_2023.csv')

# Define the years and categories you are interested in
years = ['2019', '2020', '2021', '2022', '2023']
categories = ['Current receipts, accrual basis ']

# Create a plot
plt.figure(figsize=(10,6))

# Filter and plot data for California
california_data = california_df[california_df["Description"].isin(categories)]
for index, row in california_data.iterrows():
    plt.plot(years, row[years].values, label="California", color='blue')

# Filter and plot data for Texas
texas_data = texas_df[texas_df["Description"].isin(categories)]
for index, row in texas_data.iterrows():
    plt.plot(years, row[years].values, label="Texas", color='green')

# Filter and plot data for Florida
florida_data = florida_df[florida_df["Description"].isin(categories)]
for index, row in florida_data.iterrows():
    plt.plot(years, row[years].values, label="Florida", color='red')

# Add labels and title
plt.xlabel('Year')
plt.ylabel('Income (Thousands of Dollars)')
plt.title('Income Levels Over Time for California, Texas, and Florida (2019-2023)')
plt.legend()

# Adjust the x-ticks for better readability
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()


Based on the population density and the personal income levels data extracted from the US BEA(Bureau of Economic Analysis), we can say that it would be beneficial to start a food business in California.

In the next step, we will find out which type of restaurant has the most chances of succeeding in California.

In [27]:
#SERP API Key
api_key = os.environ.get("serp_api")

In [41]:
# Define the restaurant types and California cities to search
restaurant_types = ["fast food", "casual dining", "ethnic food", "fine dining"]

# Major California cities to search in
california_cities = [
    "Los Angeles", "San Francisco", "San Diego", "Sacramento", "San Jose",
    "Fresno", "Long Beach", "Oakland", "Bakersfield", "Anaheim",
    "Santa Ana", "Riverside", "Stockton", "Irvine", "Chula Vista"
]


In [ ]:
params = {
    "engine" : "yelp",
    "find_desc" : "casual dining",
    "find_loc" : "Anaheim, California",
    "api_key" : api_key,
    "max_pages" : 2
}

search = GoogleSearch(params)
results = search.get_dict()
results
all_restaurants = []

for restaurant in results.get("organic_results"):
    restaurant_data = {
                    "name": restaurant.get("title", "N/A"),
                    "type": "Casual Dining",
                    "city": "Anaheim",
                    "rating": restaurant.get("rating", "N/A"),
                    "reviews_count": restaurant.get("reviews", "N/A"),
                    "price_range": restaurant.get("price", "N/A"),
                    "link": restaurant.get("link", "N/A"),
                    "categories": [cat.get("title", "") for cat in restaurant.get("categories", [])],
                    "address": restaurant.get("address", "N/A"),
                    "phone": restaurant.get("phone", "N/A")
    }
    all_restaurants.append(restaurant_data)
    
all_restaurants

In [42]:
# Initialize an empty list to store all restaurant data
all_restaurants_data = []

# Loop through each restaurant type and location
for restaurant_type in restaurant_types:
    for city in california_cities:
        # Update the search parameters
        params["find_desc"] = restaurant_type
        params["find_loc"] = f"{city}, California"
        
        # Perform the search
        search = GoogleSearch(params)
        results = search.get_dict()
        
        # Extract restaurant data
        for restaurant in results.get("organic_results", []):
            restaurant_data = {
                "name": restaurant.get("title", "N/A"),
                "type": restaurant_type,
                "city": city,
                "rating": restaurant.get("rating", "N/A"),
                "reviews_count": restaurant.get("reviews", "N/A"),
                "price_range": restaurant.get("price", "N/A"),
                "link": restaurant.get("link", "N/A"),
                "categories": [cat.get("title", "") for cat in restaurant.get("categories", [])],
                "address": restaurant.get("address", "N/A"),
                "phone": restaurant.get("phone", "N/A")
            }
            all_restaurants_data.append(restaurant_data)

# Convert the data into a DataFrame
restaurants_df = pd.DataFrame(all_restaurants_data)

# Export the data to a CSV file
restaurants_df.to_csv("california_restaurants_data.csv", index=False)

print("Data extraction and export completed successfully!")



Data extraction and export completed successfully!


In [1]:
ca_df = pd.read_csv('california_restaurants_data.csv')
ca_df.info()

NameError: name 'pd' is not defined

In [9]:
# Group by restaurant type and calculate average rating & review count
restaurant_type_analysis = ca_df.groupby("type").agg(
    avg_rating=("rating", "mean"),
    avg_reviews=("reviews_count", "mean"),
    count=("name", "count")  # To verify sample size
).sort_values(by=["avg_rating", "avg_reviews"], ascending=[False, False])

# Group by city and calculate the same metrics
city_analysis = ca_df.groupby("city").agg(
    avg_rating=("rating", "mean"),
    avg_reviews=("reviews_count", "mean"),
    count=("name", "count")
).sort_values(by=["avg_rating", "avg_reviews"], ascending=[False, False])

restaurant_type_analysis, city_analysis


(               avg_rating  avg_reviews  count
 type                                         
 ethnic food      4.353333   774.333333    150
 casual dining    4.344000   971.886667    150
 fine dining      4.288667  1123.053333    150
 fast food        3.448667   703.280000    150,
                avg_rating  avg_reviews  count
 city                                         
 San Diego          4.3075     1565.800     40
 Anaheim            4.2775     1369.525     40
 Long Beach         4.2725      869.925     40
 Fresno             4.2175      456.350     40
 San Francisco      4.1550     1107.575     40
 Los Angeles        4.1375     1589.850     40
 Stockton           4.1300      489.900     40
 Santa Ana          4.1225     1097.900     40
 San Jose           4.0600      750.900     40
 Irvine             4.0450      874.250     40
 Riverside          4.0450      705.425     40
 Oakland            4.0325      833.850     40
 Bakersfield        3.9575      420.250     40
 Chula Vista

In [ ]:
# Visualize average ratings and reviews by restaurant type
plt.figure(figsize=(12, 6))

# Bar plot for average ratings by restaurant type
plt.subplot(1, 2, 1)
restaurant_type_analysis["avg_rating"].plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Average Ratings by Restaurant Type")
plt.ylabel("Average Rating")
plt.xticks(rotation=45)

# Bar plot for average reviews by restaurant type
plt.subplot(1, 2, 2)
restaurant_type_analysis["avg_reviews"].plot(kind="bar", color="lightgreen", edgecolor="black")
plt.title("Average Reviews by Restaurant Type")
plt.ylabel("Average Reviews")
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Visualize average ratings and reviews by city
plt.figure(figsize=(12, 6))

# Bar plot for average ratings by city
plt.subplot(1, 2, 1)
city_analysis["avg_rating"].plot(kind="bar", color="coral", edgecolor="black")
plt.title("Average Ratings by City")
plt.ylabel("Average Rating")
plt.xticks(rotation=45)

# Bar plot for average reviews by city
plt.subplot(1, 2, 2)
city_analysis["avg_reviews"].plot(kind="bar", color="gold", edgecolor="black")
plt.title("Average Reviews by City")
plt.ylabel("Average Reviews")
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

Next step : Narrow down on a city and the type of business

Scrape the next city and business type

draw some conclusions from this one california dataset